# MNIST

In [1]:
import numpy as np
import time
import math
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score


#Hyperparameters
N_GROUPS = 10              #Number of feature groups
N_NODES = 10               #Feature nodes per group
N_ENHANCE = 11000          #Number of enhancement nodes
LAMBDA = 1e-8              #Ridge regularization parameter
BATCH_SIZE = 5000          #Mini-batch size for incremental matrix accumulation
SEED = 0                   #Random seed for reproducibility
DTYPE = np.float32         #Data type for arrays

rng = np.random.default_rng(SEED)

#Load MNIST dataset
print("Downloading MNIST dataset...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X = mnist.data.astype(DTYPE) / 255.0
y = mnist.target.astype(np.int32)

X.shape[0]
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

#One-hot encoding of labels
encoder = OneHotEncoder(sparse_output=False)
Y_train = encoder.fit_transform(y_train.reshape(-1, 1)).astype(DTYPE)
Y_test = encoder.transform(y_test.reshape(-1, 1)).astype(DTYPE)

n_features = X_train.shape[1]
n_classes = Y_train.shape[1]

#Total feature nodes and combined feature+enhancement nodes
F_TOTAL = N_GROUPS * N_NODES        #100 feature nodes
A_COLS = F_TOTAL + N_ENHANCE        #Total input dimension after feature + enhancement

#Initialize random weights and biases for feature and enhancement nodes
W_e = rng.uniform(-1, 1, size=(n_features, F_TOTAL)).astype(DTYPE)
b_e = rng.uniform(-1, 1, size=(1, F_TOTAL)).astype(DTYPE)

W_h = rng.uniform(-1, 1, size=(F_TOTAL, N_ENHANCE)).astype(DTYPE)
b_h = rng.uniform(-1, 1, size=(1, N_ENHANCE)).astype(DTYPE)

def sigmoid(z):
    #Numerically stable sigmoid function to reduce overflow warnings
    return 1 / (1 + np.exp(-z))

#Incremental computation of AtA = AᵀA and AtY = AᵀY for ridge regression
start_train = time.time()
AtA = np.zeros((A_COLS, A_COLS), dtype=DTYPE)
AtY = np.zeros((A_COLS, n_classes), dtype=DTYPE)

n_batches = math.ceil(X_train.shape[0] / BATCH_SIZE)
print(f"Building AtA and AtY matrices in {n_batches} batches...")

for i in range(n_batches):
    start_idx = i * BATCH_SIZE
    end_idx = min((i + 1) * BATCH_SIZE, X_train.shape[0])
    X_batch, Y_batch = X_train[start_idx:end_idx], Y_train[start_idx:end_idx]

    #Compute feature nodes (linear transformation)
    Z_batch = np.matmul(X_batch, W_e) + b_e

    #Compute enhancement nodes (sigmoid activation)
    H_batch = sigmoid(np.matmul(Z_batch, W_h) + b_h)

    #Concatenate feature and enhancement nodes
    A_batch = np.hstack([Z_batch, H_batch])

    #Accumulate sums for ridge regression system
    AtA += A_batch.T @ A_batch
    AtY += A_batch.T @ Y_batch

    if (i + 1) % 3 == 0 or (i + 1) == n_batches:
        print(f"Processed batch {i + 1} of {n_batches}")

#Solve ridge regression: (AtA + λI) W = AtY
print("Solving ridge regression system...")
ridge_matrix = AtA + LAMBDA * np.identity(A_COLS, dtype=DTYPE)
W_out = np.linalg.solve(ridge_matrix, AtY)

print(f"Training completed in {time.time() - start_train:.2f} seconds.")

#Inference on test set (batched)
start_test = time.time()
y_pred_batches = []

n_test_batches = math.ceil(X_test.shape[0] / BATCH_SIZE)
for i in range(n_test_batches):
    start_idx = i * BATCH_SIZE
    end_idx = min((i + 1) * BATCH_SIZE, X_test.shape[0])

    Z_test = np.matmul(X_test[start_idx:end_idx], W_e) + b_e
    H_test = sigmoid(np.matmul(Z_test, W_h) + b_h)
    A_test = np.hstack([Z_test, H_test])

    y_pred_batches.append(A_test @ W_out)

y_pred = np.vstack(y_pred_batches)

accuracy = accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_pred, axis=1))
print(f"Testing completed in {time.time() - start_test:.2f} seconds.")
print(f"Test accuracy: {accuracy * 100:.2f}%")

Building AtA and AtY matrices in 12 batches...


/tmp/ipython-input-1-1953166635.py:51: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Processed batch 3 of 12
Processed batch 6 of 12
Processed batch 9 of 12
Processed batch 12 of 12
Solving ridge regression system...
Training completed in 245.83 seconds.
Testing completed in 1.88 seconds.
Test accuracy: 96.24%


# BREAST CANCER DATASET

In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
import time

# Set random seed
np.random.seed(42)

# Hyperparameters
n_groups = 2
nodes_per_group = 8
n_enhance = 4
reg_lambda = 1e-6

# Load dataset
print("Loading Breast Cancer dataset...")
data = load_breast_cancer()
X = data.data.astype(np.float32)
y = data.target

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
Y = encoder.fit_transform(y.reshape(-1, 1)).astype(np.float32)

# Split into train/test (70/30)
split = int(len(X) * 0.7)
X_train, X_test = X[:split], X[split:]
Y_train, Y_test = Y[:split], Y[split:]

# Feature setup
n_features = X_train.shape[1]
n_classes = Y_train.shape[1]
total_feat_nodes = n_groups * nodes_per_group
input_dim = total_feat_nodes + n_enhance

# Random weights and biases
W_feat = np.random.uniform(-1, 1, (n_features, total_feat_nodes)).astype(np.float32)
b_feat = np.random.uniform(-1, 1, (1, total_feat_nodes)).astype(np.float32)

W_enh = np.random.uniform(-1, 1, (total_feat_nodes, n_enhance)).astype(np.float32)
b_enh = np.random.uniform(-1, 1, (1, n_enhance)).astype(np.float32)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Train: Compute full A, then AᵀA and AᵀY
print("Starting training...")
start = time.perf_counter()

Z_train = X_train @ W_feat + b_feat
H_train = sigmoid(Z_train @ W_enh + b_enh)
A_train = np.hstack((Z_train, H_train))

AtA = A_train.T @ A_train
AtY = A_train.T @ Y_train

# Solve ridge regression
print("Solving output weights...")
ridge = AtA + reg_lambda * np.eye(input_dim)
W_out = np.linalg.solve(ridge, AtY)

print(f"Training done in {time.perf_counter() - start:.6f} seconds")

# Test
print("Testing...")
start_test = time.perf_counter()

Z_test = X_test @ W_feat + b_feat
H_test = sigmoid(Z_test @ W_enh + b_enh)
A_test = np.hstack((Z_test, H_test))

y_pred = A_test @ W_out
acc = accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_pred, axis=1))

print(f"Test done in {time.perf_counter() - start_test:.6f} seconds")
print(f"Test Accuracy: {acc * 100:.2f}%")

Loading Breast Cancer dataset...
Starting training...
Solving output weights...
Training done in 0.003621 seconds
Testing...
Test done in 0.001937 seconds
Test Accuracy: 97.66%
